In [ ]:
import analytics.api as ga
import analytics.sheets_api as sheets
import analytics.sheets_elements as elements
import pandas as pd
import gspread
from constants import *

In [ ]:
%env ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../../../do_not_commit_ga4_credentials.json

In [ ]:
ga_authentication, drive_authentication, sheets_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    ga.sheets_service_params,
    port=OAUTH_PORT
)

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
}

lungmap_catalog_params = {
    **default_params,
    "property": LUNGMAP_ID,
}

lungmap_catalog_params_all_time = {
    **lungmap_catalog_params,
    "start_date": ANALYTICS_START,
    "end_date": END_DATE_CURRENT,
}

In [ ]:
df_monthly_pageviews = elements.get_page_views_over_time_df(lungmap_catalog_params_all_time, additional_data_path=HISTORIC_UA_DATA_PATH, additional_data_behavior=elements.ADDITIONAL_DATA_BEHAVIOR.ADD)
df_pageviews = elements.get_page_views_change(lungmap_catalog_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)
df_outbound = elements.get_outbound_links_change(lungmap_catalog_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)

In [ ]:
dict_spreadsheet = {
    "Monthly Traffic Summary": df_monthly_pageviews,
    "Pageviews": df_pageviews,
    "Outbound Links": df_outbound,
}
sheet = sheets.create_sheet_in_folder(
        drive_authentication,
        SHEET_NAME,
        PARENT_FOLDER_NAME,
        override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
    )
sheets.fill_spreadsheet_with_df_dict(
    sheet,
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    column_formatting_options={
        "Monthly Traffic Summary": {
            "Month": sheets.COLUMN_FORMAT_OPTIONS.YEAR_MONTH_DATE,
            "Users Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Pageviews Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Outbound Links": {
            "Total Clicks Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },
        "Pageviews": {
            "Total Views Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
            "Total Users Percent Change": sheets.COLUMN_FORMAT_OPTIONS.PERCENT_COLORED,
        },

    },
    sheet_formatting_options={
        "Monthly Traffic Summary": {
            "extra_columns": 1,
            "extra_columns_width": 2000
        }
    }
)
monthly_traffic_worksheet = sheet.worksheet("Monthly Traffic Summary")
date_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 1), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 2)
)
users_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 2), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 3)
)
pageviews_range = sheets.WorksheetRange(
    monthly_traffic_worksheet, 
    gspread.cell.Cell(1, 3), 
    gspread.cell.Cell(df_monthly_pageviews.index.size + 1, 4)
)
sheets.add_chart_to_sheet(
    sheets_authentication,
    sheet,
    sheet.worksheet("Monthly Traffic Summary"),
    sheets.CHART_TYPES.LINE,
    date_range,
    [users_range, pageviews_range],
    chart_position=gspread.cell.Cell(1, 6),
    chart_position_offset_x=75,
    chart_position_offset_y=75,
    title="Pageviews and Users Over Time"
)